In [52]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)


df = pd.read_csv("assets/newsherlock.csv")

data = df.to_dict('records')

In [53]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [54]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/Users/tapankarnik/Documents/3 Resources/RAG Course on Coursera/Learn-RAG/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [55]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [56]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="sherlock",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/sz/3wq8394j4g56kkvsx4sg91v80000gn/T/ipykernel_23693/3233014143.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [57]:
sliding_window_length = 5
sliding_window_step = 2
data2 = []

for i in range(0, len(data)-sliding_window_length+1, sliding_window_step):
    window = []
    for j in range(sliding_window_length):
        window.append(data[i+j]["Text"])
    window = " ".join(window)
    data2.append(dict(notes=window))




In [58]:
# vectorize!
qdrant.upload_points(
    collection_name="sherlock",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data2) # data is the variable holding all the sick one liners
    ]
)

In [59]:
user_prompt = "What did watson say at the end of the novel?"

In [60]:
# Search time for some real spirituality!

hits = qdrant.search(
    collection_name="sherlock",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
#for hit in hits:
  #print(hit.payload, "score:", hit.score)
#    pp.pprint(hit.payload)

In [61]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [62]:
reference_notes = [search_result['notes'] for search_result in search_results]
reference_notes

['enius is an infinite capacity for taking pains," he\n     remarked with a smile. "It\'s a very bad definition, but it does apply\n     to detective work."\n\n     Gregson and Lestrade had watched the manoeuvres of their amateur\n     companion with considerable curiosity and some contempt. They\n     evidently failed to appreciate the fact, which I had begun to\n     realize, that Sherlock Holmes\' smallest actions were all directed\n     towards some definite and practical end.\n\n     "What do you think of it, sir?" they both asked.\n\n     "It would be robbing you of the credit of the case if I was to\n     presume to help you," remarked my friend. "You are doing so well now\n     that it would be a pity for anyone to interfere." There was a world\n     of sarcasm in his voice as he spoke. "If you will let me know how\n     your investigations go," he continued, "I shall be happy to give you\n     any help I can. In the meantime I should like to speak to the\n     constable who fo

In [63]:
"""You are an expert in Sherlock Holmes novels. Use the following references from the supplied novel to answer my question at the end.
    Reference = """+str(reference_notes)+""" Here is the question: """+user_prompt

'You are an expert in Sherlock Holmes novels. Use the following references from the supplied novel to answer my question at the end.\n    Reference = [\'enius is an infinite capacity for taking pains," he\\n     remarked with a smile. "It\\\'s a very bad definition, but it does apply\\n     to detective work."\\n\\n     Gregson and Lestrade had watched the manoeuvres of their amateur\\n     companion with considerable curiosity and some contempt. They\\n     evidently failed to appreciate the fact, which I had begun to\\n     realize, that Sherlock Holmes\\\' smallest actions were all directed\\n     towards some definite and practical end.\\n\\n     "What do you think of it, sir?" they both asked.\\n\\n     "It would be robbing you of the credit of the case if I was to\\n     presume to help you," remarked my friend. "You are doing so well now\\n     that it would be a pity for anyone to interfere." There was a world\\n     of sarcasm in his voice as he spoke. "If you will let me know

In [64]:
# Now time to connect to the local large language model
# from openai import OpenAI
# client = OpenAI(
#     base_url="http://localhost:11434/api/generate", # "http://<Your api-server IP>:port"
#     api_key = "sk-no-key-required"
# )
# completion = client.chat.completions.create(
#     model="LLaMA_CPP",
#     messages=[
#         {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
#         {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
#         {"role": "assistant", "content": str(search_results)}
#     ]
# )
# print(completion.choices[0].message)

import requests

url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama2",
    "prompt":"""You are an expert in Sherlock Holmes novels. Use the following references from the supplied novel to answer my question at the end.
    Reference = """+str(reference_notes)+""" Here is the question: """+user_prompt,
    "stream":False
}

x = requests.post(url, json = payload)

print(x.json()['response'])

At the end of the novel, Sherlock Holmes says "It's a solved case!" This phrase signifies that he has successfully resolved the mystery and found the answers he was looking for.
